In [ ]:
import numpy as np

In [ ]:
def data_generator(range_):
    inputs = []
    targets = []
    for a in range(range_):
        v = np.random.random(2)
        targets.append(sum(v))
        inputs.append(v)
    return inputs,targets

inputs,targets = data_generator(200)
print(inputs[0],targets[0])

In [ ]:
class Model:
    def __init__(self, input, hiddens, output, learning_rate):
        self.input = input
        self.hiddens = hiddens
        self.output = output

        self.weights = []
        self.biases = []

        self.layers = [input] + hiddens + [output]

        weights = []
        biases = []
        for i in range(len(self.layers) - 1):
            weights.append(np.random.randn(self.layers[i], self.layers[i + 1]) * np.sqrt(2 / self.layers[i]))
            biases.append(np.zeros(self.layers[i + 1]))
        self.weights = weights
        self.biases = biases

        self.learning_rate = learning_rate

    def forward_pass(self, inputs):
        activations = inputs
        for index, (weight, bias) in enumerate(zip(self.weights, self.biases)):
            net_inputs = np.dot(activations, weight) + bias
            if index != len(self.weights) - 1:
                activations = self.activation(net_inputs)
            else:
                activations = net_inputs
        return activations

    def activation(self, inputs):
        return np.maximum(0, inputs)

    def activation_derivative(self, inputs):
        return (inputs > 0).astype(float)

    def backpropagation(self, error, input_):
        gradients_weights = [np.zeros_like(weight) for weight in self.weights]
        gradients_biases = [np.zeros_like(bias) for bias in self.biases]

        delta = error
        for i in range(len(self.weights) - 1, -1, -1):
            activations = self.activation(self.forward_pass(input_) if i == 0 else np.dot(self.activations[i - 1], self.weights[i - 1]))
            gradients_weights[i] = np.outer(self.activations[i], delta)
            gradients_biases[i] = delta
            delta = np.dot(delta, self.weights[i].T) * self.activation_derivative(self.activations[i])

        return gradients_weights, gradients_biases

    def train(self, epochs, inputs, targets):
        for epoch in range(epochs):
            sum_errors = 0
            for i, input_ in enumerate(inputs):
                target = targets[i]
                output = self.forward_pass(input_)
                error = target - output

                self.activations = [input_] + [self.activation(np.dot(input_, self.weights[0]) + self.biases[0])]
                for j in range(1, len(self.weights) - 1):
                    self.activations.append(self.activation(np.dot(self.activations[j], self.weights[j]) + self.biases[j]))

                gradients_weights, gradients_biases = self.backpropagation(error, input_)

                for j, (grad_weight, grad_bias) in enumerate(zip(gradients_weights, gradients_biases)):
                    self.weights[j] += self.learning_rate * grad_weight
                    self.biases[j] += self.learning_rate * grad_bias

                mean_squared_error = np.average(error ** 2)
                sum_errors += mean_squared_error

            print("Error: {} at epoch {}".format(sum_errors / len(inputs), epoch))


In [ ]:
model = Model(input=2, hiddens=[2], output=1, learning_rate=0.01)
model.train(epochs=100, inputs=inputs, targets=targets)

In [ ]:
model.forward_pass([0.2,0.3])